# Part I. ETL Pipeline for Pre-Processing the Files

## PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [7]:
# checking your current working directory
# print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#     print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [8]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [2]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## the data is in the csv - <font color=blue>event_datafile_new.csv</font>, with the columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

the data is denormalized 

### Creating a Cluster

In [4]:
# first check if cassandra server is running
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

### Create Keyspace

In [6]:
# Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    { 'class':  'SimpleStrategy', 'replication_factor':1}
    """)
except Exception as e:
    print(e)

### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exeption as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


In [80]:
# artist, song title and song's length in the music app history that was heard during 
# sessionId = 338, and itemInSession = 4
create_query = """CREATE TABLE IF NOT EXISTS music_session"""
data_types = """(sessionId int, 
            itemInSession int, 
            artist text, 
            song text, 
            song_length decimal, 
            PRIMARY KEY(sessionId, itemInSession))"""                    
query = create_query + data_types
try:
    session.execute(query)
except Exception as e:
    print(e)


In [82]:
#  read file and insert into keyspace
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO music_session (
                sessionId, 
                itemInSession, 
                artist, 
                song, 
                song_length)"""
        
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [119]:
# verify data (at primary keys in table)
query = """SELECT artist, 
        song, 
        song_length 
        FROM music_session 
        WHERE sessionId = 338 AND 
        itemInSession = 4"
        """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.artist, row.song, row.song_length)

('Unable to complete the operation against any hosts', {<Host: 127.0.0.1:9042 datacenter1>: ConnectionException('Pool is shutdown',)})


### 2. name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [87]:
# name of artist, song (sorted by itemInSession) and user (first and last name)
# for userid = 10, sessionid = 182
create_query = "CREATE TABLE IF NOT EXISTS music_user"
data_types = """(userId int, 
                sessionId int, 
                itemInSession int, 
                artist text, 
                song text, 
                firstName text, 
                lastName text, 
                PRIMARY KEY((userId), sessionId, itemInSession))"""                    
query = create_query + data_types
try:
    session.execute(query)
except Exception as e:
    
    print(e)
          

In [88]:
#  read file and insert into keyspace
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO music_user (
                userId, 
                sessionId, 
                itemInSession, 
                artist, 
                song, 
                firstName, 
                lastName)"""
        
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [97]:
# verify data (at primary keys in table)
query = """SELECT artist, 
        song, 
        firstName, 
        lastName 
        FROM music_user 
        WHERE userId = 10 
        AND sessionId = 182"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname)


Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### 3. user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [111]:
# user name (first and last) who listened to the song 'All Hands Against His Own'
create_query = """CREATE TABLE IF NOT EXISTS music_song"""
data_types = """(song text, 
                firstName text, 
                lastName text, 
                userId int, 
                PRIMARY KEY((song), userId))"""                    

query = create_query + data_types
try:
    session.execute(query)
except Exception as e:
    
    print(e)
              

In [112]:
#  read file and insert into keyspace
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO music_song ( 
                song, 
                firstName, 
                lastName, 
                userId)"""
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], line[1], line[4], int(line[10])))

In [113]:
# verify data (at primary keys in table)
query = """SELECT song, 
        firstName, 
        lastName 
        FROM music_song 
        WHERE song = 'All Hands Against His Own'"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.firstname, row.lastname)


Jacqueline Lynch
Tegan Levine
Sara Johnson


# Part III drop the tables before closing out the sessions

In [114]:
query = "DROP TABLE music_session"
query2 = "DROP TABLE music_user"
query3 = "DROP TABLE music_song"
drop_queries = [query, query2, query3]

In [115]:
for i in drop_queries:
    try:
        session.execute(i)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [116]:
session.shutdown()
cluster.shutdown()